# Experimental new pipeline

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rpy2.robjects as robjects
import multiprocessing
import re
import os

import numpy as np

from experiments.pipelines.pipeline_rpy2 import Rpy2Pipeline, Tree
from pimp.settings import getString

## Load analysis and project

In [3]:
analysis = Analysis.objects.get_or_create(id=9)[0]

DEBUG 2016-09-12 15:39:10,214 utils.py:91 | (0.000) SET SQL_AUTO_IS_NULL = 0; args=None
DEBUG 2016-09-12 15:39:10,217 utils.py:91 | (0.001) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 9 LIMIT 21; args=(9,)


In [4]:
project = Project.objects.get_or_create(id=9)[0]

DEBUG 2016-09-12 15:39:10,384 utils.py:91 | (0.000) SELECT `projects_project`.`id`, `projects_project`.`title`, `projects_project`.`user_owner_id`, `projects_project`.`description`, `projects_project`.`created`, `projects_project`.`modified` FROM `projects_project` WHERE `projects_project`.`id` = 9 LIMIT 21; args=(9,)


In [5]:
pipeline = Rpy2Pipeline(analysis, project, False)

******************************************
Setup rpy2 connection
******************************************
[1] "Initialising PuMP logging"
[1] "PuMP log level set to DEBUG"
2016-09-12 15:39:13 ERROR:pump.misc.initialisePuMPLogging:Logging at ERROR level
2016-09-12 15:39:13 WARNING:pump.misc.initialisePuMPLogging:Logging at WARNING level
2016-09-12 15:39:13 INFO:pump.misc.initialisePuMPLogging:Logging at INFO level
2016-09-12 15:39:13 DEBUG:pump.misc.initialisePuMPLogging:Logging at DEBUG level
2016-09-12 15:39:14 INFO:PeakML.Isotope.plotSamples:OS type: x86_64-apple-darwin15.5.0
2016-09-12 15:39:14 INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx512m
2016-09-12 15:39:14 INFO:PeakML.Isotope.plotSamples:Using jar file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2016-09-12 15:39:14 INFO:PeakML.Isotope.plotSamples:Using SWT file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/

In [6]:
pipeline.setup()

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: RUnit

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-12 15:39:15,023 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: RUnit

  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-12 15:39:15,057 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)



Setup called
2016-09-12 15:39:15 INFO:Pimp.getPimpWd:Data dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects
2016-09-12 15:39:15 INFO:Pimp.getPimpWd:Project dir: /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9
2016-09-12 15:39:15 INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 9
2016-09-12 15:39:15 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-12 15:39:15 9:INFO:Pimp.validateInput:Input validated correctly!
2016-09-12 15:39:15 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 9
2016-09-12 15:39:15 9:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2016-09-12 15:39:15 9:INFO:Pimp.getAnalysisParams:Setting params
2016-09-12 15:39:15 9:DEBUG:Pimp.getAnalysisParams:Name: iqr Value: 0.5
2016-09-12 15:39:15 9:DEBUG:Pimp.getAnalysisParams:Name: rsd Value: 0.5
2016-09-12 15:39:15 9:DEBUG:Pimp.getAnalysisParams:Name: noise Value: 0.8
2016-09-12 15:39:15 9:DEBUG:Pimp.getAnalysisParams:Name: p

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_9

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-12 15:39:15,268 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_9

  res = super(Function, self).__call__(*new_args, **new_kwargs)



2016-09-12 15:39:15 9:INFO:Pimp.createAnalysisDir:Analysis dir created at analysis_9


## Peak detection

In [7]:
def get_value(r_vect, key):
    return r_vect[r_vect.names.index(key)]

In [8]:
def get_formatted_value(r_vect, key, analysis_id=None, subst=None):

    str_vect = get_value(r_vect, key)
    my_str = str_vect[0]
    
    # replace 'analysis' with 'analysis_%d'
    if analysis_id is not None:
        my_str = my_str.replace('analysis', 'analysis_%d' % analysis_id)
    
    # replace the 'polarity' bit
    if subst is not None:
        my_str = my_str % subst

    return my_str

In [9]:
def peak_detection(files, xcms_params, n_slaves):

    # collect the parameters
    args = {
        'files'           : files,
        'method'          : get_value(xcms_params, 'method'),
        'ppm'             : get_value(xcms_params, 'ppm'),
        'peakwidth'       : get_value(xcms_params, 'peakwidth'),
        'snthresh'        : get_value(xcms_params, 'snthresh'),
        'prefilter'       : get_value(xcms_params, 'prefilter'),
        'integrate'       : get_value(xcms_params, 'integrate'),
        'mzdiff'          : get_value(xcms_params, 'mzdiff'),
        'verbose.columns' : get_value(xcms_params, 'verbose.columns'),
        'fitgauss'        : get_value(xcms_params, 'fitgauss'),
        'nSlaves'         : n_slaves
    }

    # call centwave
    xcms_set = robjects.r['xcmsSet']    
    xset = xcms_set(**args)
    return xset

In [10]:
positive_files = get_value(pipeline.metadata.r_files, 'positive')
xcms_params = get_value(pipeline.pimp_params, 'xcms.params')
n_slaves = multiprocessing.cpu_count()-1

print positive_files, xcms_params, n_slaves

 [1] "samples/POS/Beer_4_full1.mzXML" "samples/POS/Beer_4_full2.mzXML"
 [3] "samples/POS/Beer_4_full3.mzXML" "samples/POS/Beer_1_full1.mzXML"
 [5] "samples/POS/Beer_1_full2.mzXML" "samples/POS/Beer_1_full3.mzXML"
 [7] "samples/POS/Beer_2_full1.mzXML" "samples/POS/Beer_2_full2.mzXML"
 [9] "samples/POS/Beer_2_full3.mzXML" "samples/POS/Beer_3_full1.mzXML"
[11] "samples/POS/Beer_3_full2.mzXML" "samples/POS/Beer_3_full3.mzXML"
 $method
[1] "centWave"

$ppm
[1] 3

$peakwidth
[1]   5 100

$snthresh
[1] 2

$prefilter
[1]   2 500

$integrate
[1] 1

$mzdiff
[1] 0.001

$verbose.columns
[1] TRUE

$fitgauss
[1] FALSE

 7


In [11]:
xset_pos = peak_detection(positive_files, xcms_params, n_slaves)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘snow’


  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-12 15:39:21,242 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘snow’


  res = super(Function, self).__call__(*new_args, **new_kwargs)

/Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:BiocGenerics’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, clusterSplit, parApply, parCapply,
    parLapply, parRapply, parSapply


  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2016-09-12 15:39:21,245 functions.py:106 | /Users/joewandy/git/pimp/venv/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The f

Starting snow cluster with 7 local sockets.
Detecting features in file # 1 : Beer_4_full1.mzXML 
Detecting features in file # 2 : Beer_4_full2.mzXML 
Detecting features in file # 3 : Beer_4_full3.mzXML 
Detecting features in file # 4 : Beer_1_full1.mzXML 
Detecting features in file # 5 : Beer_1_full2.mzXML 
Detecting features in file # 6 : Beer_1_full3.mzXML 
Detecting features in file # 7 : Beer_2_full1.mzXML 
Detecting features in file # 8 : Beer_2_full2.mzXML 
Detecting features in file # 9 : Beer_2_full3.mzXML 
Detecting features in file # 10 : Beer_3_full1.mzXML 
Detecting features in file # 11 : Beer_3_full2.mzXML 
Detecting features in file # 12 : Beer_3_full3.mzXML 


## Peak alignment

In [12]:
mzmatch_params = get_value(pipeline.pimp_params, 'mzmatch.params')
peakml_params = get_value(pipeline.pimp_params, 'peakml.params')
mzmatch_outputs = get_value(pipeline.pimp_params, 'mzmatch.outputs')
method = get_formatted_value(mzmatch_params, 'rt.alignment')
print method

obiwarp


In [13]:
print mzmatch_params

$rsd
[1] 0.5

$iqr
[1] 0.5

$rt.alignment
[1] "obiwarp"

$noise
[1] 0.8

$ppm
[1] 3

$order
[1] 3

$maxrt
[1] 120

$rtwindow
[1] 30

$id.rtwindow
[1] 0.05

$combination
[1] "set"

$mindetections
[1] 3

$minintensity
[1] 5000

$adducts.positive
[1] "M+H,M+ACN+Na,M+Na,M+K,M+ACN+H"

$adducts.negative
[1] "M-H,M-ACN+Na,M-Na,M-K,M-ACN+H"




In [14]:
print mzmatch_outputs

$alignment.folder
[1] "analysis/%s/combined_rt.alignment"

$combined.folder
[1] "analysis/%s/combined"

$combined.rsd.filtered.folder
[1] "analysis/%s/combined_RSD_filtered"

$combined.rsd.rejected.folder
[1] "analysis/%s/combined_RSD_rejected"

$final.combined.peakml.file
[1] "analysis/%s/final_combined.peakml"

$final.combined.noise.filtered.file
[1] "analysis/%s/final_combined_nf.peakml"

$final.combined.simple.filtered.file
[1] "analysis/%s/final.combined_sf.peakml"

$final.combined.gapfilled.file
[1] "analysis/%s/final_combined_gapfilled.peakml"

$final.combined.related.file
[1] "analysis/%s/final_combined_related.peakml"

$final.combined.basepeaks.file
[1] "analysis/%s/final_combined_basepeaks.peakml"

$final.combined.related.id.file
[1] "analysis/%s/final_combined_related_identified.peakml"

$final.combined.related.stds.id.file
[1] "analysis/%s/final_combined_related_stds_identified.peakml"

$final.combined.basepeaks.id.file
[1] "analysis/%s/final_combined_basepeaks_identified.p

In [15]:
method = get_value(mzmatch_params, 'rt.alignment')[0]

In [16]:
xset = xset_pos

In [17]:
def rt_correct(xset, method, peakml_params, mzmatch_outputs, n_slaves, verbose):
    
    if method == 'obiwarp':
        retcor = robjects.r['retcor']
        xset_aln = retcor(xset, method='obiwarp', profStep=0.01)        
    elif method == 'loess':
        retcor = robjects.r['retcor']
        xset_aln = retcor(xset, method='loess', family="symmetric")
    else:
        xset_aln = xset
        
    filepaths = robjects.r['filepaths']
    fp = filepaths(xset_aln)

    split = robjects.r['split']
    xseto = split(xset_aln, fp)
    return fp, xseto

In [18]:
fp, xseto = rt_correct(xset, method, peakml_params, mzmatch_outputs, n_slaves, True)

center sample:  Beer_1_full2 
Processing: Beer_4_full1  Beer_4_full2  Beer_4_full3  Beer_1_full1  Beer_1_full3  Beer_2_full1  Beer_2_full2  Beer_2_full3  Beer_3_full1  Beer_3_full2  Beer_3_full3  


#### get java heapsize

In [19]:
xmx = getString('PIMP_JAVA_PARAMETERS', '')
print xmx

found = re.findall(r'\d+', xmx)
heapsize = int(found[0])
print heapsize

-Xmx8192m
8192


#### initialize mzmatch

In [20]:
# I think this should be called only once??
args = {
    'memorysize' : heapsize,
    'version.1'  : False
}
mzmatch_init = robjects.r['mzmatch.init']
mzmatch_init(**args)

2016-09-12 15:49:41 9:INFO:PeakML.Isotope.plotSamples:OS type: x86_64-apple-darwin15.5.0
2016-09-12 15:49:41 9:INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx8192m
2016-09-12 15:49:41 9:INFO:PeakML.Isotope.plotSamples:Using jar file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2016-09-12 15:49:41 9:INFO:PeakML.Isotope.plotSamples:Using SWT file /Users/joewandy/git/pimp/packrat/lib/x86_64-apple-darwin15.5.0/3.3.1/mzmatch.R/java/swt-4.5-cocoa-macosx-x86_64.jar


<BoolVector - Python:0x125517b90 / R:0x7f9c2b304bf8>
[       0]

In [21]:
write_peakml = robjects.r['PeakML.xcms.write.SingleInstance']

In [22]:
polarity_folder = get_formatted_value(mzmatch_outputs, 'polarity.folder', analysis_id=analysis.id, subst='pos')

In [23]:
ionisation = get_value(peakml_params, 'ionisation')
add_scans = get_value(peakml_params, 'addscans')
write_rejected = get_value(peakml_params, 'writeRejected')
apodisation_filter = get_value(peakml_params, 'ApodisationFilter')
ppm = get_value(peakml_params, 'ppm') # not set inside peakml_params ..?
if ppm == 0:
    ppm = 5

In [24]:
peakml_files = []
regex = re.compile(re.escape('mzxml'), re.IGNORECASE)
for single_fp in fp:
    replaced = regex.sub('peakml', single_fp)
    outfile = os.path.join(polarity_folder, os.path.basename(replaced))
    peakml_files.append(outfile)

In [25]:
print peakml_files

['analysis_9/pos/Beer_4_full1.peakml', 'analysis_9/pos/Beer_4_full2.peakml', 'analysis_9/pos/Beer_4_full3.peakml', 'analysis_9/pos/Beer_1_full1.peakml', 'analysis_9/pos/Beer_1_full2.peakml', 'analysis_9/pos/Beer_1_full3.peakml', 'analysis_9/pos/Beer_2_full1.peakml', 'analysis_9/pos/Beer_2_full2.peakml', 'analysis_9/pos/Beer_2_full3.peakml', 'analysis_9/pos/Beer_3_full1.peakml', 'analysis_9/pos/Beer_3_full2.peakml', 'analysis_9/pos/Beer_3_full3.peakml']


In [26]:
directory = os.path.dirname(peakml_files[0])
if not os.path.exists(directory):
    os.makedirs(directory)

In [27]:
for i in range(len(peakml_files)):
    print 'Now creating %s' % peakml_files[i]
    write_peakml(xseto[i], outputfile=peakml_files[i], ionisation=ionisation, 
                 addscans=add_scans, writeRejected=write_rejected, 
                 ApodisationFilter=apodisation_filter, ppm=ppm)

Now creating analysis_9/pos/Beer_4_full1.peakml
2016-09-12 15:49:42 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-12 15:49:42 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2016-09-12 15:49:42 9:INFO:PeakML.xcms.write.SingleInstance:- /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/samples/POS/Beer_1_full1.mzXML
2016-09-12 15:50:04 9:INFO::11539 peaks exported.
Now creating analysis_9/pos/Beer_4_full2.peakml
2016-09-12 15:50:04 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-12 15:50:04 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2016-09-12 15:50:04 9:INFO:PeakML.xcms.write.SingleInstance:- /Users/joewandy/git/pimp/django_projects/pimp_data/projects/9/samples/POS/Beer_1_full2.mzXML
2016-09-12 15:50:29 9:INFO::11471 peaks exported.
Now creating analysis_9/pos/Beer_4_full3.peakml
2016-09-12 15:50:30 9:INFO:PeakML.xcms.write.SingleInstance:positive
2016-09-12 15:50:30 9:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data

## Combine

Combine files by group, RSD filter if required (Not convinced we should be using this filter) and combine group files

In [28]:
combined_dir = os.path.join(directory, 'combined')

In [29]:
if not os.path.exists(combined_dir):
    os.makedirs(combined_dir)

In [30]:
factors = pipeline.get_groups()

In [31]:
print factors

[beer_sample with 4 levels, beer_colour with 2 levels, beer_taste with 3 levels]


In [32]:
dims = []
for f in factors:
    print f.label
    for lev in f.levels:
        print ' -', lev, f.level_files[lev]
    dims.append(len(f.levels))

beer_sample
 - beer1 ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3']
 - beer2 ['Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3']
 - beer3 ['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3']
 - beer4 ['Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']
beer_colour
 - dark ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3']
 - light ['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3', 'Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']
beer_taste
 - delicious ['Beer_1_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_2_full1', 'Beer_2_full2', 'Beer_2_full3']
 - okay ['Beer_3_full1', 'Beer_3_full2', 'Beer_3_full3']
 - awful ['Beer_4_full1', 'Beer_4_full2', 'Beer_4_full3']


In [33]:
all_pos_files = pipeline.get_short_names('positive')

In [39]:
def populate_tree(parent, factors):
    depth = parent.depth + 1
    if depth > len(factors)-1:
        return
    factor = factors[depth]
    nodes = []
    for level in factor.levels:
        node = Tree(factor, level, depth, parent)
        populate_tree(node, factors)
        
def print_tree(node):
    output = ''.join(['\t' for i in range(node.depth)])
    output += '%d %s %s : %s\n' % (node.depth, node.factor, node.level_label, node.level_files)
    for child in node.children:
        output += print_tree(child)
    return output

parent = Tree(None, 'root', -1, None)
parent.level_files = set(pipeline.get_short_names('positive'))
populate_tree(parent, factors)
output = print_tree(parent)
print output

-1 None root : set(['Beer_3_full1', 'Beer_1_full2', 'Beer_1_full3', 'Beer_3_full2', 'Beer_1_full1', 'Beer_3_full3', 'Beer_4_full3', 'Beer_4_full2', 'Beer_4_full1', 'Beer_2_full1', 'Beer_2_full3', 'Beer_2_full2'])
0 beer_sample with 4 levels beer1 : set(['Beer_1_full2', 'Beer_1_full3', 'Beer_1_full1'])
	1 beer_colour with 2 levels dark : set(['Beer_1_full2', 'Beer_1_full3', 'Beer_1_full1'])
		2 beer_taste with 3 levels delicious : set(['Beer_1_full2', 'Beer_1_full3', 'Beer_1_full1'])
		2 beer_taste with 3 levels okay : set([])
		2 beer_taste with 3 levels awful : set([])
	1 beer_colour with 2 levels light : set([])
		2 beer_taste with 3 levels delicious : set([])
		2 beer_taste with 3 levels okay : set([])
		2 beer_taste with 3 levels awful : set([])
0 beer_sample with 4 levels beer2 : set(['Beer_2_full1', 'Beer_2_full3', 'Beer_2_full2'])
	1 beer_colour with 2 levels dark : set(['Beer_2_full1', 'Beer_2_full3', 'Beer_2_full2'])
		2 beer_taste with 3 levels delicious : set(['Beer_2_full1'

In [ ]:
mat = np.empty(dims, dtype=object)
print mat.shape

In [ ]:
beer_sample = groups[0]
mat[0, :, :] = beer_sample.level_files[beer_sample.levels[0]]
print mat[0]

In [ ]:
for i in range(len(groups)):
    d = dims[i]
    mat[d, :, :] = 

In [ ]:
mat.shape

In [ ]:
print mat

In [ ]:
mat[0, 0] = 'a'

In [ ]:
print mat